In [1]:
# downloading the data
# !gdown --id 1zcDOotHJ3tvYE67ecR8fJrM1sXv-elLH


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
data = pd.read_csv(r"C:\Users\Administrator\Downloads\amazon.csv")
data

reviewText  Positive
0      This is a one of the best apps acording to a b...         1
1      This is a pretty good version of the game for ...         1
2      this is a really cool game. there are a bunch ...         1
3      This is a silly game and can be frustrating, b...         1
4      This is a terrific game on any pad. Hrs of fun...         1
...                                                  ...       ...
19995  this app is fricken stupid.it froze on the kin...         0
19996  Please add me!!!!! I need neighbors! Ginger101...         1
19997  love it!  this game. is awesome. wish it had m...         1
19998  I love love love this app on my side of fashio...         1
19999  This game is a rip off. Here is a list of thin...         0

[20000 rows x 2 columns]

# exploratory data analysis

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   reviewText  20000 non-null  object
 1   Positive    20000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 312.6+ KB


In [5]:
data.describe()

Positive
count  20000.000000
mean       0.761650
std        0.426085
min        0.000000
25%        1.000000
50%        1.000000
75%        1.000000
max        1.000000

In [6]:
# printing th efirst 10 values of the data
for i in range(len(data)):
    if i == 10:
        break
    else:
        print(data.iloc[int(i),0])

This is a one of the best apps acording to a bunch of people and I agree it has bombs eggs pigs TNT king pigs and realustic stuff
This is a pretty good version of the game for being free. There are LOTS of different levels to play. My kids enjoy it a lot too.
this is a really cool game. there are a bunch of levels and you can find golden eggs. super fun.
This is a silly game and can be frustrating, but lots of fun and definitely recommend just as a fun time.
This is a terrific game on any pad. Hrs of fun.  My grandkids love it. Great entertainment when waiting in long lines
This is a very entertaining game!  You don't have to be smart to play it.  I guess that's why I like it...it's easy and fun and that's what games are suppose to be.  Be warned: this game is highly addictive.
this is awesome and you don't need wi ti to play trust me. it is really fun and addicting. there are like 100 levels it is even free don't waste your money on the expensive one I mean seriously. get the app
this

In [7]:
data.loc[:,"Positive"].value_counts()

Positive
1    15233
0     4767
Name: count, dtype: int64

---

# Data visuallization

### sentiment analysis plot

In [8]:
from bokeh.io import output_file, show
from bokeh.plotting import figure
from bokeh.transform import factor_cmap
from bokeh.models import ColumnDataSource, FactorRange, HoverTool
from bokeh.palettes import Spectral6
import pandas as pd


# Count occurrences of positive and negative sentiment
sentiment_counts = data['Positive'].value_counts().sort_index()
sentiment_labels = ['Negative', 'Positive']  # 0 -> Negative, 1 -> Positive

# Prepare Bokeh data source
source = ColumnDataSource(data=dict(sentiment=sentiment_labels, counts=sentiment_counts))

# Output to an HTML file that will open in a new browser window
output_file("sentiment_distribution.html")

# Create a figure with toolbar tools for dynamic interaction
p = figure(x_range=sentiment_labels, outer_height=400, outer_width=600, title="Sentiment Distribution",
           toolbar_location="above", tools="pan,wheel_zoom,box_zoom,reset,hover", 
           tooltips=[("Sentiment", "@sentiment"), ("Count", "@counts")],
           x_axis_label="Sentiment", y_axis_label="Frequency")

# Add vertical bars
p.vbar(x='sentiment', top='counts', width=0.4, source=source, 
       fill_color=factor_cmap('sentiment', palette=['#e84d60', '#6ecb74'], factors=sentiment_labels))

# Customize the plot for better appearance
p.title.text_font_size = "16pt"
p.xgrid.grid_line_color = None
p.y_range.start = 0
p.xaxis.major_label_text_font_size = "12pt"
p.yaxis.major_label_text_font_size = "12pt"

# Enable dynamic features like zoom, pan, hover
hover = p.select(dict(type=HoverTool))
hover.tooltips = [("Sentiment", "@sentiment"), ("Count", "@counts")]

# Show the plot in a new browser window
show(p)


In [9]:
# !pip install wordcloud

# word cloud

In [10]:
from wordcloud import WordCloud
from bokeh.io import output_file, show
from bokeh.plotting import figure
from PIL import Image
import pandas as pd
import numpy as np

#
# Generate Word Cloud for Positive Sentiment
positive_text = ' '.join(data[data['Positive'] == 1]['reviewText'])

# Create WordCloud image
wordcloud = WordCloud(width=800, height=400, background_color='white', colormap='Set3').generate(positive_text)

# Save the wordcloud to a file
wordcloud_image_path = 'positive_wordcloud.png'
wordcloud.to_file(wordcloud_image_path)

# Load the image using PIL and convert to RGBA format
img = Image.open(wordcloud_image_path).convert("RGBA")
w, h = img.size

# Convert image to NumPy array (RGBA)
image_array = np.array(img)

# Flip the image vertically because Bokeh uses a different coordinate system
image_array = np.flipud(image_array)

# Output to an HTML file
output_file("positive_wordcloud.html")

# Create a figure with full-screen settings
p = figure(x_range=(0, w), y_range=(0, h), width=w, height=h, 
           title="Positive Sentiment Word Cloud", toolbar_location=None, tools="")

# Hide grid and axis
p.grid.visible = False
p.axis.visible = False

# Convert RGBA values from 8-bit integers to 32-bit integers for Bokeh
image_rgba = np.dstack((image_array[:, :, 0], image_array[:, :, 1], image_array[:, :, 2], image_array[:, :, 3])).astype(np.uint32)

# Shift the bits to match Bokeh's RGBA format
image_rgba = (image_rgba[:, :, 0] << 24) | (image_rgba[:, :, 1] << 16) | (image_rgba[:, :, 2] << 8) | image_rgba[:, :, 3]

# Display the image
p.image_rgba(image=[image_rgba], x=0, y=0, dw=w, dh=h)

# Show the plot in a new browser window
show(p)


# text length

In [11]:
# third 

from bokeh.io import output_file, show
from bokeh.plotting import figure
from bokeh.layouts import gridplot


# Calculate the length of each text
data['text_length'] = data['reviewText'].apply(len)

# Separate text lengths based on sentiment
positive_lengths = data[data['Positive'] == 1]['text_length']
negative_lengths = data[data['Positive'] == 0]['text_length']

# Output to a new browser window
output_file("text_length_distribution_separate_plots.html")

# Create histogram bins
bins = np.linspace(0, max(data['text_length']), 20)

# Positive sentiment plot
positive_hist, edges = np.histogram(positive_lengths, bins=bins)
p1 = figure(title="Positive Sentiment Text Length Distribution", height=400, width=500,
           x_axis_label="Text Length", y_axis_label="Frequency", toolbar_location=None, tools="")
p1.quad(top=positive_hist, bottom=0, left=edges[:-1], right=edges[1:], 
        fill_color="green", alpha=0.6)
p1.xgrid.grid_line_color = None
p1.ygrid.grid_line_color = None

# Negative sentiment plot
negative_hist, edges = np.histogram(negative_lengths, bins=bins)
p2 = figure(title="Negative Sentiment Text Length Distribution", height=400, width=500,
           x_axis_label="Text Length", y_axis_label="Frequency", toolbar_location=None, tools="")
p2.quad(top=negative_hist, bottom=0, left=edges[:-1], right=edges[1:], 
        fill_color="red", alpha=0.6)
p2.xgrid.grid_line_color = None
p2.ygrid.grid_line_color = None

# Create a grid layout for both plots side by side
grid = gridplot([[p1, p2]])

# Show the plots in a new browser window
show(grid)
